In [ ]:
!pip install openai

In [2]:
import os
import json
import csv
from openai import OpenAI
import os
from time import sleep

In [ ]:
OPENAI_API_KEY="YourAPIKey"

In [4]:
def convert_csv_to_json(inputfile,outputfile):
    system_message={"role":"system","message":"you are a helpful assistant."}

    with open(inputfile,'r',encoding='utf-8') as inputcsv,\
         open(outputfile,'w',encoding='utf-8') as ouputcsv:
        
        input_reader=csv.reader(inputcsv)
        next(input_reader) # to skip the first header

        for row in input_reader:
            medical_report=row[0]
            output_format =row[1]

            # print('medical report:',medical_report)
            # print('output format:',output_format)

            training_example={"messages" :
                                [system_message,
                                {"role":"user",    "message":medical_report},
                                {"role":"assistant","message":output_format}
                                ]

                              }
            ouputcsv.write(json.dumps(training_example) + '\n')

    

In [5]:
convert_csv_to_json('medical_records_50.csv','training_data.jsonl')
convert_csv_to_json('validation_medical_records_10.csv','validation_data.jsonl')

In [6]:
llm=OpenAI(api_key=OPENAI_API_KEY)

In [7]:
#upload training file
def upload_training_file(input_file):

    with open(input_file,'rb') as file:
        response=llm.files.create(file=file,
                                  purpose="fine-tune")
        return response.id
    



In [8]:
training_file_id=upload_training_file("training_data.jsonl")
validation_file_id=upload_training_file("validation_data.jsonl")
print('Training File Id           :',training_file_id)
print('validation_file_id File Id :',validation_file_id)

Training File Id           : file-CGw5JvKyqt7P9Uc1Wji6uj
validation_file_id File Id : file-TKGENenDNSroT8VMYVkBB4


In [9]:
#create fine tuning job
def create_fine_tuning_job(model,training_file,validation_file=None):
    response=llm.fine_tuning.jobs.create(training_file=training_file,
                                         validation_file=validation_file,
                                         model=model)
    return response.id


In [10]:
#Execute Fine Tuning Job id

model='gpt-4o-mini-2024-07-18'
job_id=create_fine_tuning_job(model,training_file_id,validation_file_id)
print('Job id :',job_id)

Job id : ftjob-M7zQyPRFIY4NOsLAUxcLY3xL


In [11]:
#monitor job

def monitor_job(job_id):
    while True:
        job=llm.fine_tuning.jobs.retrieve(job_id)
        print(f"Job Running Status {job.status}")

        if job.status in ['succeeded','failed']:
            return job
        
        # List latest events
        events = llm.fine_tuning.jobs.list_events(
            fine_tuning_job_id=job_id,
            limit=5
        )
        for event in events.data:
            print(f"Event: {event.message}")
        
        sleep(30)



In [12]:
job=monitor_job(job_id)
if job.status == "succeeded":
    fine_tuned_model = job.fine_tuned_model
    print(f"Fine-tuned model ID: {fine_tuned_model}")
else:
    print("Fine-tuning failed.")

Job Running Status validating_files
Event: Validating training file: file-CGw5JvKyqt7P9Uc1Wji6uj and validation file: file-TKGENenDNSroT8VMYVkBB4
Event: Created fine-tuning job: ftjob-M7zQyPRFIY4NOsLAUxcLY3xL
Job Running Status running
Event: The job failed due to a file format error in the training file. Invalid file format. Example 1, message 1 is missing key "content".
Event: Validating training file: file-CGw5JvKyqt7P9Uc1Wji6uj and validation file: file-TKGENenDNSroT8VMYVkBB4
Event: Created fine-tuning job: ftjob-M7zQyPRFIY4NOsLAUxcLY3xL
Job Running Status failed
Fine-tuning failed.


In [ ]:
def test_model(model_id, test_input):
    """Test the fine-tuned model"""
    completion = llm.chat.completions.create(
        model=model_id,
        messages=[
            {
                "role": "system",
                "content": "Extract Details from medical report"
            },
            {"role": "user", "content": test_input}
        ]
    )
    return completion.choices[0].message


In [ ]:
# Test input
test_report = """Marcus Wong, a 19-year-old male, presents with severe acne 
on face and upper back present for 1 year. Multiple inflammatory papules 
and nodules noted on examination. Previous trials of over-the-counter 
treatments ineffective. Started on Isotretinoin 40mg daily with monthly 
liver function monitoring."""

# Get prediction
result = test_model(fine_tuned_model, test_report)

# Parse the JSON response
import json
extracted_data = json.loads(result.content)
print(json.dumps(extracted_data, indent=2))
